<a href="https://colab.research.google.com/github/NAWTTY/khomol/blob/master/ML_Individual_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 1. Mount Drive




Firstly we have to mount the drive from google cola.




Also importing the google drive where the file is uploaded and stored.

In [ ]:
from google.colab import drive
drive.mount('/gdrive')


# 2. Path MyDrive

You can instruct IPython to change its own working directory by typing cd or%cd.
 
This will continue during your IPython session, but it will not change the shell from which you started IPython. 

That shell will still be in the directory it was in when you began IPython after you end IPython.

In [ ]:
%cd '/gdrive/MyDrive/'

# 3. Pip Installation 


!pip installation of command that will perform most basic functions i.e the installation of particular analysis tools

Through useful visuals, missingno offers the ability to comprehend the distribution of missing values. 

Others are panda filing, empiricaldist which disturb the empirical function and factor analyzer to analyze exploratory and factor analysis (EFA)

In [ ]:
!pip install missingno
!pip install pandas-profiling
!pip install empiricaldist
!pip install factor-analyzer

# 4. Loading analysis tools




loading required analysis tools which i.e panda, nump, mateplotlib,seaborn which are the specified tools for the operation.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

import missingno as msno
%matplotlib inline

pd.set_option('display.max_columns', 100)

# 5. Loading data 

loading data which are the train and test files that are uploaded to colab drive or Mydrive

*   here we can fetch the dataset and also read the information of individual listed on file
*   This files are sorted using panda to call up the csv files



In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Recognisation of first dataset

In [ ]:
train.head()

In [ ]:
train.tail()

In [ ]:
train.info()

In [ ]:
print("Train dataset (rows, cols):",train.shape, "\nTest dataset (rows, cols):",test.shape)

In [ ]:
import pandas_profiling as pdp

In [ ]:
#@title Additional format
import os
from os import listdir
import pandas as pd
import numpy as np
import glob
import tqdm
from typing import Dict
import matplotlib.pyplot as plt
%matplotlib inline
!pip install chart_studio
import plotly.express as px
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

In [ ]:
train.columns

In [ ]:
data = []
for feature in train.columns:
    # Defining the role
    if feature == 'target':
        role = 'target'
    elif feature == 'id':
        role = 'id'
    else:
        role = 'input'
         
    # Defining the level
    if 'bin' in feature or feature == 'target':
        level = 'binary'
    elif 'cat' in feature or feature == 'id':
        level = 'nominal'
    elif train[feature].dtype == float:
        level = 'interval'
    else:
        level = 'ordinal'
        
    # Initialize keep to True for all variables except for id
    keep = True
    if feature == 'id':
        keep = False
    
    # Defining the data type 
    dtype = train[feature].dtype
    
    # Creating a Dict that contains all the metadata for the variable
    feature_dict = {
        'varname': feature,
        'role': role,
        'level': level,
        'keep': keep,
        'dtype': dtype
    }
    data.append(feature_dict)
    
meta1 = pd.DataFrame(data, columns=['varname', 'role', 'level', 'keep', 'dtype'])
meta1.set_index('varname', inplace=True)
meta1

In [ ]:
data = []
for feature in train.columns:
    # Defining the role
    if feature == 'target':
        use = 'target'
    elif feature == 'id':
        use = 'id'
    else:
        use = 'input'
         
    # Defining the type
    if 'bin' in feature or feature == 'target':
        type = 'binary'
    elif 'cat' in feature or feature == 'id':
        type = 'categorical'
    elif train[feature].dtype == float or isinstance(train[feature].dtype, float):
        type = 'real'
    else:
        type = 'integer'
        
    # Initialize preserve to True for all variables except for id
    preserve = True
    if feature == 'id':
        preserve = False
    
    # Defining the data type 
    dtype = train[feature].dtype
    
    category = 'none'
    # Defining the category
    if 'ind' in feature:
        category = 'individual'
    elif 'reg' in feature:
        category = 'registration'
    elif 'car' in feature:
        category = 'car'
    elif 'calc' in feature:
        category = 'calculated'
    
    
    # Creating a Dict that contains all the metadata for the variable
    feature_dictionary = {
        'varname': feature,
        'use': use,
        'type': type,
        'preserve': preserve,
        'dtype': dtype,
        'category' : category
    }
    data.append(feature_dictionary)
    
meta2 = pd.DataFrame(data, columns=['varname', 'use', 'type', 'preserve', 'dtype', 'category'])
meta2.set_index('varname', inplace=True)
meta2

In [ ]:
meta2[(meta2.type == 'categorical') & (meta2.preserve)].index

In [ ]:
pd.DataFrame({'count' : meta2.groupby(['category'])['category'].size()}).reset_index()

In [ ]:
pd.DataFrame({'count' : meta2.groupby(['use', 'type'])['use'].size()}).reset_index()

# Exploratory Data Analysis (EDA) <a name='eda'></a>

## Data Quality Issues <a name='data_quality'></a>

### Data duplications

In [ ]:
train.drop_duplicates()
train.shape

In [ ]:
test.drop_duplicates()
test.shape

Observations:
* **Train** has 595,212 rows and 59 variables
* 1 one missing variable in the **Test** (58 variables) 
* no duplication in both **Train** and **Test** dataset

### Missing Values
Missing values represented by -1 can be replaced with NaN. Most of the classifiers have pretty good strategies to manage missing (or NaN) values.

In [ ]:
def _impute_missing_data(data):
    return data.replace(-1, np.nan)

train = _impute_missing_data(train)
test = _impute_missing_data(test)

In [ ]:
print('NaN values =', train.isnull().sum().sum())
print("""""")

vars_with_missing = []

for feature in train.columns:
    missings = train[feature].isna().sum()
    
    if missings > 0 :
        vars_with_missing.append(feature)
        missings_perc = missings / train.shape[0]
        
        print('Variable {} has {} records ({:.2%}) with missing values.'.format(feature, missings, missings_perc))
print('In total, there are {} variables with missing values'.format(len(vars_with_missing)))

We can use **missingno** package to visualise the missing values. To do that, we must first replace the -1 values into NAN value, then use  missingno.
Note: if missingno has not been previously installed, you can install it using the following command within Jupyter-notebook: 
**!pip install missingno**

In [ ]:
import missingno as msno  # Visualize missing values
%matplotlib inline
msno.matrix(train)

In [ ]:
msno.bar(train);

Observations:
- **ps_car_03_cat and ps_car_05_cat** have a large proportion of  records with missing values. Remove these variables.
- For the other categorical variables with missing values, we can leave the missing value -1 as such.
- **ps_reg_03** (real/interval) has missing values for 18% of all records. Replace by the mean.
- **ps_car_11** (categorical/ordinal) has only 5 records with misisng values. Replace by the mode.
- **ps_car_12** (real/interval) has only 1 records with missing value. Replace by the mean.
- **ps_car_14** (real/interval) has missing values for 7% of all records. Replace by the mean.

In [ ]:
msno.heatmap(train)

Observations:
- **ps_ind_01_cat/ps_ind_02_cat/ps_ind_04_cat** have the same missing values 
- missing value relationship between **ps_car_03_cat/ps_ind_02_cat** seems very similar to **ps_car_03_cat/ps_ind_04_cat**
- missing value relationship between **ps_car_07_cat/ps_ind_02_cat** seems very similar to **ps_car_07_cat/ps_ind_04_cat** and **ps_car_07_cat/ps_car_01_cat and ps_ind_05_cat/ps_ind_02_cat** and **ps_ind_05_cat/ps_ind_04_cat** 
- Negative relationships may suggest imputational opportunities are they signify cases where missing values exist in one variable but not in another 

In [ ]:
df_missing_train = pd.DataFrame({'column':train.columns, 'missing(%)':((train.isna()).sum()/train.shape[0])*100})
df_missing_test = pd.DataFrame({'column':train.columns, 'missing(%)':((train.isna()).sum()/train.shape[0])*100})

df_missing_train_nl = df_missing_train.nlargest(7, 'missing(%)')
df_missing_test_nl = df_missing_test.nlargest(7, 'missing(%)')

sns.set_palette(sns.color_palette('nipy_spectral'))

plt.figure(figsize=(16,6))
sns.barplot(data= df_missing_train_nl, x='column', y='missing(%)',palette='nipy_spectral')
plt.title('Missing values (%) in training set')
plt.show()

plt.figure(figsize=(16,6))
sns.barplot(data= df_missing_test_nl, x='column', y='missing(%)',palette='nipy_spectral')
plt.title('Missing values (%) in test set')
plt.show()

Observations: Missing value proportions tend to be consistent across the **train** and **test** dataset

## Univariate Exploration <a name='univariate_exploration'></a>

### Binary features

Let's plot the histogram of **target** column within the **train** dataset

In [ ]:
sns.set_style('white')
sns.set(font_scale=1)
plt.figure()
sns.countplot(x=train['target'],palette='nipy_spectral')
plt.show()

 Let's obtain get the descriptive the binary features within the  **train** dataset

In [ ]:
v = meta2[(meta2.type == 'binary') & (meta2.preserve)].index
train[v].describe()

Observations:

- The mean value of **target** in the **train** dataset is 3.6448%, which is **strongly imbalanced** between zeros and ones.
- From the means we can conclude that for most variables the value is zero in most cases.

Let's plot the distribution of the binary data in the **train** dataset,  with **blue** represent the percent of 0, and the **red** represent the percent of 1

In [ ]:
bin_col = [col for col in train.columns if '_bin' in col]
zero_list = []
one_list = []
for col in bin_col:
    zero_list.append((train[col]==0).sum()/train.shape[0]*100)
    one_list.append((train[col]==1).sum()/train.shape[0]*100)
plt.figure()
fig, ax = plt.subplots(figsize=(6,6))
# Bar plot
p1 = sns.barplot(ax=ax, x=bin_col, y=zero_list, color="blue")
p2 = sns.barplot(ax=ax, x=bin_col, y=one_list, bottom= zero_list, color="red")
plt.ylabel('Percent of zero/one [%]', fontsize=12)
plt.xlabel('Binary features', fontsize=12)
locs, labels = plt.xticks()
plt.setp(labels, rotation=90)
plt.tick_params(axis='both', which='major', labelsize=12)
plt.show();

Observations: **ps_ind_10_bin**, **ps_ind_11_bin**, **ps_ind_12_bin** and **ps_ind_13_bin** have very small number of values 1 (lesss than 0.5%) whilst the number of value 1 is very large for **ps_ind_16_bin** and **ps_cals_16_bin** (more than 60%).



Let's see the distribution of **binary** variables and the corresponding values of **target** variable.

In [ ]:
var = meta2[(meta2.type == 'binary') & (meta2.preserve)].index
var = [col for col in train.columns if '_bin' in col]
i = 0
t1 = train.loc[train['target'] != 0]
t0 = train.loc[train['target'] == 0]

sns.set_style('whitegrid')
plt.figure()
fig, ax = plt.subplots(5,4,figsize=(16,16))

for feature in var:
    i += 1
    plt.subplot(5,4,i)
    sns.kdeplot(t1[feature], bw_adjust=20)
    sns.kdeplot(t0[feature], bw_adjust=20)
    plt.ylabel('Density plot', fontsize=12)
    plt.xlabel(feature, fontsize=12)
    locs, labels = plt.xticks()
    plt.tick_params(axis='both', which='major', labelsize=12)
    plt.legend(['targe=1', 'target=0'], loc='best')
plt.show();

### Categorical (Nominal) features

Let's see the distribution of **categorical** variables and the corresponding values of **target** variable.

In [ ]:
var = meta2[(meta2.type == 'categorical') & (meta2.preserve)].index
i = 0
t1 = train.loc[train['target'] != 0]
t0 = train.loc[train['target'] == 0]

sns.set_style('whitegrid')
plt.figure()
fig, ax = plt.subplots(4,4,figsize=(16,16))

for feature in var:
    i += 1
    plt.subplot(4,4,i)
    sns.kdeplot(t1[feature], bw_adjust=20)
    sns.kdeplot(t0[feature], bw_adjust=20)
    plt.ylabel('Density plot', fontsize=12)
    plt.xlabel(feature, fontsize=12)
    locs, labels = plt.xticks()
    plt.tick_params(axis='both', which='major', labelsize=12)
    plt.legend(['targe=1', 'target=0'], loc='best')
plt.show();

We calculate the percentage of target=1 per category value and represent these percentages using bar plots (histogram).

In [ ]:
var = meta2[(meta2.type == 'categorical') & (meta2.preserve)].index

for feature in var:
    fig, ax = plt.subplots(figsize=(6,6))
    # Calculate the percentage of target=1 per category value
    cat_perc = train[[feature, 'target']].groupby([feature],as_index=False).mean()
    cat_perc.sort_values(by='target', ascending=False, inplace=True)
    # Bar plot
    # Order the bars descending on target mean
    sns.barplot(ax=ax,x=feature, y='target', data=cat_perc, order=cat_perc[feature])
    plt.ylabel('Percent of target with value 1 [%]', fontsize=12)
    plt.xlabel(feature, fontsize=12)
    plt.tick_params(axis='both', which='major', labelsize=12)
    plt.show();

Here, we check the cardinality of the categorical variables. Cardinality refers to the number of different values in a variable. 

In [ ]:
var = meta2[(meta2.type == 'categorical') & (meta2.preserve)].index

for feature in var:
    dist_values = train[feature].value_counts().shape[0]
    print('Variable {} has {} distinct values'.format(feature, dist_values))

Observations: Only **ps_car_11_cat** has many distinct values, although it is still reasonable.

### Real (Interval) features

In [ ]:
variable = meta2[(meta2.type == 'real') & (meta2.preserve)].index
train[variable].describe()

Observation:

**reg**: The range (min to max) differs between the variables. We could apply scaling (e.g. StandardScaler), but it depends on the classifier we will want to use

**car**: Again, the range differs and we could apply scaling

**calc**: This seems to be some kind of ratio as the maximum is 0.9. All three *_calc* variables have very similar distributions

**Overall**, we can see that the range of the interval variables is rather small. Perhaps some transformation (e.g. log) is already applied in order to anonymise the data?


## Bivariate Exploration <a name='bivariate_exploration'></a>

### Real (Interval) features

Let's visualise the relationship between features in the Real (Interval) variables using Correlation 
Matrix

In [ ]:
def corr_heatmap(sample, masking=False):
    sns.set_style('whitegrid')
    # Create color map ranging between two colors
    cmap = sns.diverging_palette(50, 10, as_cmap=True)
    fig, ax = plt.subplots(figsize=(10,10))
    
    if masking==False:
        correlations = sample.corr()
        sns.heatmap(correlations, cmap=cmap, vmax=1.0, center=0, fmt='.2f', 
                    square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .75})
    else:
        correlations = np.triu(sample.corr())
        sns.heatmap(sample.corr(), cmap=cmap, vmax=1.0, center=0, fmt='.2f',
                square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .75}, 
                    mask=correlations)
    plt.show();    

In [ ]:
sample = train.sample(1000)
var = meta2[(meta2.type == 'real') & (meta2.preserve)].index
sample = sample[var]
corr_heatmap(sample)

Observations - The variables with strong correlations are:
- ps_reg_02 and ps_reg_03
- ps_car_12 and ps_car13
- ps_car_12 and ps_car14
- ps_car_13 and ps_car15

In [ ]:
corr_heatmap(sample, masking=True)

Let's visualise the features with strong correlations in the Real (Interval) variables using Pairplot 

In [ ]:
sample = train.sample(1000)
var = ['ps_reg_01', 'ps_reg_02', 
       'ps_reg_03', 'ps_car_12', 
       'ps_car_13', 'ps_car_14',
       'ps_car_15', 'target']
sample = sample[var]
sns.pairplot(sample,  hue='target', 
             palette = 'Set1', diag_kind='kde')
plt.show()

Let's visualise the correlation between 2 specific features in the Real (Interval) variables using linear model plot (lmplot)

In [ ]:
sns.lmplot(x='ps_reg_02', y='ps_reg_03', data=sample, 
           hue='target', palette='Set1', 
           scatter_kws={'alpha':0.3})
plt.show()

In [ ]:
sns.lmplot(x='ps_car_15', y='ps_car_13', data=sample, 
           hue='target', palette='Set1', 
           scatter_kws={'alpha':0.3})
plt.show()

In [ ]:
sns.lmplot(x='ps_car_12', y='ps_car_13', data=sample, 
           hue='target', palette='Set1', 
           scatter_kws={'alpha':0.3})
plt.show()

In [ ]:
sns.lmplot(x='ps_car_12', y='ps_car_14', data=sample, 
           hue='target', palette='Set1', 
           scatter_kws={'alpha':0.3})
plt.show()

Let's visualise the correlation between 2 specific features in the Real (Interval) variables using Joint-Plot

In [ ]:
sns.jointplot(x='ps_reg_02', y='ps_reg_03', data=sample, 
              kind="hist")

In [ ]:
sns.jointplot(x='ps_car_15', y='ps_car_13', data=sample, 
              kind="hist")

In [ ]:
sns.jointplot(x='ps_car_12', y='ps_car_13', data=sample, 
              kind="hist")

In [ ]:
sns.jointplot(x='ps_car_12', y='ps_car_14', data=sample, 
              kind="hist")

In [ ]:
sample_nc = train.sample(1000)
var_nc = ['ps_calc_01', 'ps_calc_02', 'ps_calc_03', 'target']
sample_nc = sample_nc[var_nc]

sns.lmplot(x='ps_calc_01', y='ps_calc_02', data=sample_nc, 
           hue='target', palette='Set1', scatter_kws={'alpha':0.3})
sns.lmplot(x='ps_calc_01', y='ps_calc_03', data=sample_nc, 
           hue='target', palette='Set1', scatter_kws={'alpha':0.3})

sns.jointplot(x='ps_calc_01', y='ps_calc_02', data=sample_nc, kind="hist")
sns.jointplot(x='ps_calc_02', y='ps_calc_03', data=sample_nc, kind="hist")

plt.show()

Question: How can we decide which of the correlated variables to keep?

### Integer (Ordinal) features

Let's visualise the relationship between features in the Integer (Ordinal) variables using Correlation Matrix


In [ ]:
sample = train.sample(1000)
var = meta2[(meta2.type == 'integer') & (meta2.preserve)].index
sample = sample[var]
corr_heatmap(sample)

Observations: For the *integer* variables we do not see many correlations.

In [ ]:
def corr_heatmap1(sample, method):
    sns.set_style('whitegrid')
    # Create color map ranging between two colors
    cmap = sns.diverging_palette(50, 10, as_cmap=True)
    fig, ax = plt.subplots(figsize=(10,10))
    correlations = sample.corr(method)
    sns.heatmap(correlations, cmap=cmap, vmax=1.0, center=0, fmt='.2f', 
                square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .75})
    plt.show();    

In [ ]:
corr_heatmap1(sample, 'kendall')
corr_heatmap1(sample, 'spearman')

In [ ]:
sns.jointplot(x='ps_ind_01', y='ps_ind_03', data=sample, kind="hist")
plt.show()

In [ ]:
sns.jointplot(x='ps_ind_01', y='ps_calc_04', data=sample, kind="hist")
plt.show()

Let's visualise the probabilities of the possible values for a variable using Probability Mass Function (PMF). PMF requires empiricaldist package. If not available, you may tnstall empiricaldist using the following command line: <br>
!pip install empiricaldist

In [ ]:
from empiricaldist import Cdf, Pmf
train_pmf = train.sample(n = 500)
fig, ax = plt.subplots()
# Extract the unique categories
categories = train_pmf['ps_ind_02_cat'].unique()
for cat in categories:
    pmf_cat = Pmf.from_seq(train_pmf[
        train_pmf['ps_ind_02_cat'] == cat]['ps_car_12'])
    ax.plot(pmf_cat, label=cat)
ax.set(xlabel='ps_car_12',
       ylabel='P(X = x)')
plt.title('The number of categories: {}'.format(len(categories)))
ax.legend()
plt.show();

Let's visualise the distribution between 2 specific features The ‘boxenplot’ is a nonparametric representation of a distribution in which all features correspond to actual observations. By plotting more quantiles, it provides more information about the shape of the distribution, particularly in the tails. 

In [ ]:
sns.boxenplot(x=train_pmf.ps_car_14.astype('float'), 
              y=train_pmf.ps_car_10_cat.astype('category'))

In [ ]:
train_clustermap = train.sample(n = 50)
int_features = train_clustermap.select_dtypes(include=['int64']).columns.tolist()
ordinal_features = [o for o in int_features if 
                    ('cat' not in o and 'bin' not in o and 'id' not in o and 'target' not in o )]
ord_features_df = train_clustermap[ordinal_features]
sns.clustermap(ord_features_df)

## Conclusion
I hope  this notebook helped you learn the exploratory data analysis (EDA) workflow, that is an important initial steps before proceeding to the data preprocessing for the ML model development.